<a href="https://colab.research.google.com/github/joyMajumder123/Banking-website/blob/main/Extreme_Weather_Forecasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'widsdatathon2023:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F43454%2F4872828%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T154344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1208f33bf279ad8c34f0cb6df7ff2fad13d4dcb0678bf53e95fab4f4c12640dccbdb64fd280201b20d4c54f80dfb076ce06c3eefbfff6a1e92cb289a0bab375501d4bd51b5b980f25561633d83f3b534d13ca6b3e03b0b74a2612aa516908d04592e0b64dddc87b90594e6fe224dd1fe827f4cad78524a04d9c5cf4f5ce11fe3cfbbf15bdf229c8eeed2ff3bd53f13cdcd068ffde15000702c080ad2ab7b9c5a61a5d61e24b5153bb53208d2891ad7414321cc60b41b9e4f8e16027180e3f96da58e80d816af9bff29233136b2caba4275af5adff2d0baecbd1a010e2a25a3a019d8283fe93d169cf57681b5cfe942621d10ff8cfc06ba36373e4f7ec5e3fb79'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 213184469 bytes downloaded
Downloaded and uncompressed: widsdatathon2023
Data source import complete.


# Women in Data Science (WiDS Datathon) 2023
## Adapting to Climate Change by Improving Extreme Weather Forecasts with Machine Learning

#### The most of research studies show climate change has led to future warming, and devastating weather is growing more frequent. Extreme weather events event and conditions such as heatwaves, cold spells, heavy precipitation events, and tropical and extratropical cyclones take a significant toll on worldwide human lives, public health, economic prosperity, infrastructure, and ecosystems. Therefore, it is crucial to improve our understanding of extreme weather events and to predict their impacts through accurate frameworks for predicting extreme weather events ahead of time.
#### Observing, monitoring, and predicting extreme events and managing their impacts requires an extraordinary amount of information about the state of Earth and how it changes from moment to moment and decade to decade. There are many differences between the various types of extreme weather events. Predicting them requires regional and seasonal knowledge about each component of the climate system, including ocean, atmosphere, sea ice, etc.
#### This dataset consists of enormous number of variables related to  Temperature, Precipitation, Sea surface temperature and sea ice concentration, Multivariate ENSO index (MEI), Madden-Julian oscillation (MJO),  Relative humidity, sea level pressure, and precipitable water for the entire atmosphere, Geopotential height, zonal wind, and longitudinal wind, North American Multi-Model Ensemble (NMME), Pressure and potential evaporation, Elevation, and Köppen-Geiger climate classifications.

1. [What to forecast](#1)
2. [DataSet](#2)
3. [Load the Data](#3)
4. [View Data](#4)
5. [Handeling missing Data](#5)
6. [Analyse the Target Variable](#6)
7. [Exploratory Data Analysis](#7)
8. [Train | Validation Split](#8)
9. [Time Series Split for Cross Validation](#9)
10. [Training, Evaluating and Validation](#10)

<a id="1"></a> <br>
# 1. What to forecast/Predict

The WiDS Datathon 2023 focuses on a prediction task involving forecasting sub-seasonal temperatures (temperatures over a two-week period) within the United States.

<a id="2"></a> <br>
# 2. DataSet

The dataset has 256 variables, including the unique index and the target variable **"contest-tmp2m-14d__tmp2m"**.
The data are from 514 locations within 15 climate zones in the United States.
The train dataset includes daily data instances between 09/01/2014 - 08/31/2016
The dataset includes daily data instances between 11/01/2022 - 12/31/2022 and expected to predict the temperature variable contest-tmp2m-14d__tmp2m.  

![1.jpg](attachment:f3dfefa6-aee6-447c-aef3-aa569b200e03.jpg)/)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import norm
from scipy import stats
import math

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
pip install -U feature-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.3/344.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [ ]:
from feature_engine.creation import CyclicalFeatures
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import DropFeatures

<a id="3"></a> <br>
# 3. Load the Data

In [ ]:
train_data = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv', index_col = ['startdate'])
test_data = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv', index_col = ['startdate'])
submit = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')

<a id="4"></a> <br>
# 4. View Data

## First, have a look in to the Train dataset

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
# Check the data types and data info
train_data.info(verbose=True, show_counts=True)

**Most of the variables are numerical. Right now, only one variable is categorical. It is 244 features and the target variable "contest-tmp2m-14d__tmp2m". That's a lot**

In [ ]:
test_data.info(verbose=True, show_counts=True)

In [ ]:
train_data.shape, test_data.shape

**Test data does not include data for the target variable 'contest-tmp2m-14d__tmp2m'. All the other variables which are in the train data set are in the test data**

In [ ]:
train_data.describe().transpose()

In [ ]:
# cast train dataset startdate variable into date format
train_data.index = pd.to_datetime(train_data.index)
train_data.index

In [ ]:
train_data.index.min(), train_data.index.max()

**Simply it is continuous daily data for 245 variables. First thought, this is a complex dataset. Maximum Timestamp is ('2014-09-01 00:00:00'), and the minimum Timestamp is ('2016-08-31 00:00:00')). It should be 731
Data instances. But we have 375734 data instances. It seems the dataset stacked several datasets on each other. Let's check...**

In [ ]:
# cast test dataset startdate variable into date format
test_data.index = pd.to_datetime(test_data.index)
test_data.index

In [ ]:
test_data.index.min(), test_data.index.max()

**We have test data for 61 days time period. But in total we have 31354 data instances**

## Simplify the dataset complexity by using the "lon" and "lat" variables

In [ ]:
# Create a new variable called 'location' by using 'lon' & 'lat'
import math
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

train_data['trunc_lon'] = train_data['lon'].apply(lambda x: truncate(x,4))
train_data['trunc_lon'] = train_data['trunc_lon'].map('{:.9f}'.format)

train_data['trunc_lat'] = train_data['lat'].apply(lambda x: truncate(x,4))
train_data['trunc_lat'] = train_data['trunc_lat'].map('{:.9f}'.format)

In [ ]:
cols = ['trunc_lon', 'trunc_lat']
train_data['location'] = train_data[cols].apply(lambda row: '_'.join(row.values.astype(object)), axis=1)

In [ ]:
train_data['location'].describe()

**Problem solved! The dataset belongs to 514 unique locations. 514 * 731 daily data instances. That is how it made for 375734 data instances**

### Do the same thing for the test dataset

In [ ]:
test_data['trunc_lon'] = test_data['lon'].apply(lambda x: truncate(x,4))
test_data['trunc_lon'] = test_data['trunc_lon'].map('{:.9f}'.format)

test_data['trunc_lat'] = test_data['lat'].apply(lambda x: truncate(x,4))
test_data['trunc_lat'] = test_data['trunc_lat'].map('{:.9f}'.format)

In [ ]:
test_data['location'] = test_data[cols].apply(lambda row: '_'.join(row.values.astype(object)), axis=1)

In [ ]:
test_data['location'].describe()

**The dataset belongs to 514 unique locations. 514 * 61 daily data instances. That is how it made for 31354 data instances. But there is a problem again, are they belong to the same locations as in the train dataset or different?**

In [ ]:
# Drop unnecessary columns from the train data frame
train_data.drop(['index','lon','lat','trunc_lon','trunc_lat'], axis=1, inplace=True)

In [ ]:
train_data.head()

In [ ]:
# Drop unnecessary columns from the test data frame
test_data.drop(['index','lon','lat','trunc_lon','trunc_lat'], axis=1, inplace=True)

In [ ]:
test_data.head()

<a id="5"></a> <br>
# 5. Handeling missing Data

### Check for missing data in Train Dataset

In [ ]:
#quick check for missing data
#null_counts=train_cleaned.isnull().sum()
#data_null = null_counts[null_counts > 0]

def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

missing_zero_values_table(train_data)

In [ ]:
# Trying to identify the correlation between the features which have missing values
msno.heatmap(train_data, cmap="RdYlGn", figsize=(10,5), fontsize=10)

**There are 8 columns that have missing values.
Looks like they have good correlation between each other**

**Just look for their distribution**

In [ ]:
# Thinking about the zero values possible for these variables.
fig, axes = plt.subplots(2,4, figsize=(15, 5))
ax = axes.flatten()
zero_list = ['nmme0-tmp2m-34w__ccsm30','nmme0-prate-56w__ccsm30', 'nmme0-prate-34w__ccsm30', 'ccsm30',
             'nmme-tmp2m-56w__ccsm3','nmme-prate-56w__ccsm3', 'nmme-prate-34w__ccsm3', 'nmme-tmp2m-34w__ccsm3']
for i, col in enumerate(zero_list):
    sns.histplot(train_data[col], ax=ax[i])
    ax[i].set_title(col)

    ax[i].ticklabel_format(style='plain', axis='both')

fig.tight_layout(w_pad=6, h_pad=4)
plt.show()

**The variables are precipitation, and temperature related. With my knowledge, it is possible to have zero values for them. So I decided to leave them as it is**

**I decided to fill in Null values at this stage. But it might cause data leakage problems**

**I used the dataframe method fillna for filling missing values,which will propagate last valid observation forward to next valid.
Since the missing value percentage is comparatively small, I hope this will not be a big deal for this dataset**  

In [ ]:
# train_cleaned.interpolate(method ='linear', limit_direction ='forward')
train_data.fillna(method = 'ffill' , inplace = True)

### Check for missing data in Test Dataset

In [ ]:
# quick check for missing data in test dataset
null_counts=test_data.isnull().sum()
null_counts[null_counts > 0]

**Glad! No any missing data in test dataset**

<a id="6"></a> <br>
# 6. Analyse the Target Variable 'contest-tmp2m-14d__tmp2m'

In [ ]:
train_data['contest-tmp2m-14d__tmp2m'].describe()

In [ ]:
# Distribution plot
plt.figure(figsize=(8,5))
sns.set() # for style
sns.distplot(train_data['contest-tmp2m-14d__tmp2m'] , fit=norm)
plt.title("Histogram of contest-tmp2m-14d__tmp2m") # for histogram title
# probability plot
plt.figure(figsize=(8,5))
res = stats.probplot(train_data['contest-tmp2m-14d__tmp2m'], plot=plt)
plt.show()

# skewness and kurtosis
print("Skewness: %f" % train_data['contest-tmp2m-14d__tmp2m'].skew())
print("Kurtosis: %f" % train_data['contest-tmp2m-14d__tmp2m'].kurt())

The skewness of 'contest-tmp2m-14d__tmp2m' was found to be -0.227, indicating that the distribution was little left-skewed.

The kurtosis of 'contest-tmp2m-14d__tmp2m' was found to be -0.517, indicating that the distribution lighter-tailed compared to the normal distribution.

**Do we need to take any actions ? As I think we don't need...**

<a id="7"></a> <br>
# 7. Exploratory Data Analysis

## Visualize Time-series Data

In [ ]:
# Visualize time-series data Temperature and Precipitation
variables = list(train_data.columns)
for var in ['contest-tmp2m-14d__tmp2m', 'contest-precip-14d__precip']:
    #plot the time series
    train_data[var].plot(figsize=(20,4))

    # add title
    plt.title(var)

    #the y axis lable
    plt.ylabel(var)

    plt.show()

## Seasonality - Temperature

In [ ]:
# Capture "date" in a new variable
time_m = train_data.index.month
time_m =pd.Series(time_m, index=train_data.index)
time_y = train_data.index.year
time_y =pd.Series(time_y, index=train_data.index)

In [ ]:
train_data.groupby(time_m)[['contest-tmp2m-14d__tmp2m']].mean().plot(figsize =(15,5))
plt.title("Temperature seasonality - monthly(mean value from all 514 locations)")
plt.ylabel("Temperature")

In [ ]:
train_data.groupby(time_y)[['contest-tmp2m-14d__tmp2m']].mean().plot(figsize =(15,5))
plt.title("Temperature seasonality - yearly(mean value from all 514 locations)")
plt.ylabel("Temperature")

The mean temperature values are increasing between 2014 and 2015. It is nearly **6 degrees within 2 years!**

## Seasonality - Precipitation

In [ ]:
train_data.groupby(time_m)[['contest-precip-14d__precip']].mean().plot(figsize =(15,5))
plt.title("Precipitation seasonality - monthly(mean value from all 514 locations)")
plt.ylabel("Precipitation")

We can see the temperature variation throughout the year. During summer it is going to high values, and during winter it is getting low.

In [ ]:
train_data.groupby(time_y)[['contest-precip-14d__precip']].mean().plot(figsize =(15,5))
plt.title("Precipitation seasonality - yearly(mean value from all 514 locations)")
plt.ylabel("Precipitation")

## Look Deeper in Precipitation Forcasting

In [ ]:
precipitation_cols = [col for col in train_data if 'prate' in col ]
precipitation_subset  = train_data[[col for col in train_data if 'prate' in col ]]
print(precipitation_cols)

In [ ]:
# Compute corr matrix for weighted average of monthly/most recent monthly NMME model forecasts for precipitation
corr = precipitation_subset.corr()
plt.figure(figsize=(15, 15))

sns.heatmap(corr[(corr >= 0.9) | (corr <= -0.9)],
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

There is a very high correlation between 'gfdlflora', and 'gfdlflorb' values, and also we can see a few other variables. We can drop them from the data frame.  


In [ ]:
cancm3_col = ['nmme-prate-34w__cancm3','nmme-prate-56w__cancm3','nmme0-prate-34w__cancm30','nmme0-prate-56w__cancm30','contest-precip-14d__precip']
train_data.groupby(time_m)[cancm3_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for cancm3 - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
cancm4_col = ['nmme-prate-34w__cancm4','nmme-prate-56w__cancm4','nmme0-prate-34w__cancm40','nmme0-prate-56w__cancm40','contest-precip-14d__precip']
train_data.groupby(time_m)[cancm4_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for cancm4 - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
ccsm3_col = ['nmme-prate-34w__ccsm3','nmme-prate-56w__ccsm3','nmme0-prate-34w__ccsm30','nmme0-prate-56w__ccsm30','contest-precip-14d__precip']
train_data.groupby(time_m)[ccsm3_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for ccsm3 - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
ccsm4_col = ['nmme-prate-34w__ccsm4','nmme-prate-56w__ccsm4','nmme0-prate-34w__ccsm40','nmme0-prate-56w__ccsm40','contest-precip-14d__precip']
train_data.groupby(time_m)[ccsm4_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for ccsm4 - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
cfsv2_col = ['nmme-prate-34w__cfsv2','nmme-prate-56w__cfsv2','nmme0-prate-34w__cfsv20','nmme0-prate-56w__cfsv20','contest-precip-14d__precip']
train_data.groupby(time_m)[cfsv2_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for cfsv2 - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
gfdlflora_col = ['nmme-prate-34w__gfdlflora','nmme-prate-56w__gfdlflora','nmme0-prate-34w__gfdlflora0',
                 'nmme0-prate-56w__gfdlflora0','contest-precip-14d__precip']
train_data.groupby(time_m)[gfdlflora_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for gfdlflora - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
gfdlflorb_col = ['nmme-prate-34w__gfdlflorb','nmme-prate-56w__gfdlflorb','nmme0-prate-34w__gfdlflorb0',
                 'nmme0-prate-56w__gfdlflorb0','contest-precip-14d__precip']
train_data.groupby(time_m)[gfdlflorb_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for gfdlflorb - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
nasa_col = ['nmme-prate-34w__nasa','nmme-prate-56w__nasa','nmme0-prate-34w__nasa0',
            'nmme0-prate-56w__nasa0','contest-precip-14d__precip']
train_data.groupby(time_m)[nasa_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation forecasts for nasa - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

In [ ]:
nmmemean_col = ['nmme-prate-34w__nmmemean','nmme-prate-56w__nmmemean','nmme0-prate-34w__nmme0mean',
            'nmme0-prate-56w__nmme0mean','contest-precip-14d__precip']
train_data.groupby(time_m)[nmmemean_col].mean().plot(figsize =(15,5))
plt.suptitle("monthly NMME model precipitation average forecast across those models - Mean values for all 514 locations")
plt.title("along with Measured Precipitation")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.ylabel("precipitation")

## Madden–Julian oscillation(MJO)

The Madden–Julian oscillation (MJO) is the largest element of the intraseasonal (30- to 90-day) variability in the tropical atmosphere. It is a large-scale coupling between atmospheric circulation and tropical deep atmospheric convection. Unlike a standing pattern like the El Niño–Southern Oscillation (ENSO), the Madden–Julian oscillation is a traveling pattern that propagates eastward, at approximately 4 to 8 m/s (14 to 29 km/h; 9 to 18 mph), through the atmosphere above the warm parts of the Indian and Pacific oceans. This overall circulation pattern manifests itself most clearly as anomalous rainfall. (Source - Wikipedia).

There are a number of ways in which the MJO influences world weather: (source - metoffice.gov.uk)

* The MJO creates favourable conditions for tropical cyclone activity, which makes the MJO important to monitor during the Atlantic hurricane season.
* The enhanced rainfall phase of the MJO can also bring the onset of the Monsoon seasons around the globe. Conversely, the suppressed convection phase can delay the onset of the Monsoon season.
* There is evidence that the MJO influences the El Nino Southern Oscillation (ENSO) cycle. It does not cause El Nino or La Nina, but it can contribute to the speed of development and intensity of El Nino and La Nina episodes. The MJO appears to be more active during neutral and weak ENSO years.
* There is also evidence to suggest that the MJO can influence the onset of a Sudden Stratospheric Warming (SSW) event.

In [ ]:
# Look for any correlation between these interesting variables with Measured Precipitation
interesting_subset  = train_data[['mjo1d__phase', 'mjo1d__amplitude', 'mei__mei','mei__meirank','mei__nip', 'contest-pevpr-sfc-gauss-14d__pevpr',
                      'contest-wind-h10-14d__wind-hgt-10', 'contest-rhum-sig995-14d__rhum', 'contest-wind-h100-14d__wind-hgt-100',
                      'contest-slp-14d__slp', 'contest-wind-vwnd-925-14d__wind-vwnd-925', 'contest-pres-sfc-gauss-14d__pres',
                      'contest-wind-uwnd-250-14d__wind-uwnd-250','contest-prwtr-eatm-14d__prwtr', 'contest-wind-vwnd-250-14d__wind-vwnd-250',
                      'contest-tmp2m-14d__tmp2m', 'contest-wind-h850-14d__wind-hgt-850', 'contest-wind-uwnd-925-14d__wind-uwnd-925',
                      'contest-wind-h500-14d__wind-hgt-500']]
corr = interesting_subset.corrwith(train_data['contest-precip-14d__precip']).abs().sort_values(ascending=False)
print(corr)

In [ ]:
# Look for any correlation between these interesting variables and the target variable (Observed Temperature)
interesting_subset  = train_data[['mjo1d__phase', 'mjo1d__amplitude', 'mei__mei','mei__meirank','mei__nip', 'contest-pevpr-sfc-gauss-14d__pevpr',
                      'contest-wind-h10-14d__wind-hgt-10', 'contest-rhum-sig995-14d__rhum', 'contest-wind-h100-14d__wind-hgt-100',
                      'contest-slp-14d__slp', 'contest-wind-vwnd-925-14d__wind-vwnd-925', 'contest-pres-sfc-gauss-14d__pres',
                      'contest-wind-uwnd-250-14d__wind-uwnd-250','contest-prwtr-eatm-14d__prwtr', 'contest-wind-vwnd-250-14d__wind-vwnd-250',
                      'contest-precip-14d__precip', 'contest-wind-h850-14d__wind-hgt-850', 'contest-wind-uwnd-925-14d__wind-uwnd-925',
                      'contest-wind-h500-14d__wind-hgt-500']]
corr = interesting_subset.corrwith(train_data["contest-tmp2m-14d__tmp2m"]).abs().sort_values(ascending=False)
print(corr)
high_corr = corr.index[corr>0.75]
print(high_corr)

It shows Madden–Julian oscillation (MJO) amplitude, phase considerably correlated with temperature more than the rainfall in given locations.

(**wind at geopotential height at 100 millibars**)'contest-wind-h100-14d__wind-hgt-100', (**wind at geopotential height at 500 millibars**)'contest-wind-h500-14d__wind-hgt-500', (**pressure**)'contest-pevpr-sfc-gauss-14d__pevpr', (**potential evaporation**)'contest-prwtr-eatm-14d__prwtr' **have very high correlations with the target variable**.

In [ ]:
corr = interesting_subset .corr()
plt.figure(figsize=(15, 15))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.5)],
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

how ever (**wind at geopotential height at 100 millibars**)'contest-wind-h100-14d__wind-hgt-100', (**wind at geopotential height at 500 millibars**)'contest-wind-h500-14d__wind-hgt-500' has a very high correlation between each other. So, it could not be very useful to keep both variables for training our model.

In [ ]:
# Madden-Julian Oscillation (MJO) amplitude, phase and MEI rank, Niño Index Phase
for var in [ 'contest-precip-14d__precip','contest-tmp2m-14d__tmp2m','mjo1d__phase', 'mjo1d__amplitude', 'mei__mei','mei__meirank','mei__nip']:
    #plot the time series
    train_data.groupby(time_m)[[var]].mean().plot(figsize =(15,2))

    # add title
    plt.title(var)

    #the y axis lable
    plt.ylabel(var)

    plt.show()

## Climate Regions

![Koppen_climate_types_USA.jpg](attachment:76f671f2-24c9-4f99-b6bf-ad38a198e561.jpg)

Source(Internet)

In [ ]:
climate_regions = np.array(pd.Categorical(train_data['climateregions__climateregion']).categories)
climate_regions

In [ ]:
plt.figure(figsize=(15, 5))

y1 = train_data['climateregions__climateregion'].value_counts()
y2 = test_data['climateregions__climateregion'].value_counts()

X_axis = np.arange(len(climate_regions))

plt.bar(X_axis - 0.2, y1, 0.4, label = 'train_data')
plt.bar(X_axis + 0.2, y2, 0.4, label = 'test_data')

plt.xticks(X_axis, climate_regions)
plt.xlabel("Climate Region")
plt.ylabel("Count")
plt.title("Climate Regions Data Distribution in in each Dataset")
plt.grid()
plt.legend()
plt.show()

###  Check the temperature deviations in each climate zone based on the year

In [ ]:
ClimateR_temp = train_data.groupby([train_data.index.year,
                                    'climateregions__climateregion']).agg({'contest-tmp2m-14d__tmp2m':[np.min, np.max, np.mean,np.std]})
ClimateR_temp.columns = ['Minimum','Maximum','Mean','STD']
ClimateR_temp.reset_index(inplace=False)
ClimateR_temp

**Here I checked the yearly values. In the same way we can check the monthly values. This data will be useful for our predictions. We can roughly conclude the accuracy of our test predictions based on this**

<a id="8"></a> <br>
# 8. Train | Validation Split

In [ ]:
train_set = train_data[train_data.index <= '2016-03-31']
val_set = train_data[train_data.index > '2016-03-31'] # 2016-03-31
train_set.shape, val_set.shape

In [ ]:
X_train = train_set.drop("contest-tmp2m-14d__tmp2m", axis=1).copy(deep=True)
y_train = train_set["contest-tmp2m-14d__tmp2m"].copy(deep=True)
X_train.shape, y_train.shape

In [ ]:
X_valid = val_set.drop("contest-tmp2m-14d__tmp2m", axis=1).copy(deep=True)
y_valid = val_set["contest-tmp2m-14d__tmp2m"].copy(deep=True)
X_valid.shape, y_valid.shape

In [ ]:
X_train.head()

In [ ]:
X_valid.head()

In [ ]:
X_train.index.min(), X_train.index.max()

In [ ]:
X_valid.index.min(), X_valid.index.max()

## Data Preparation

### Exploring and Handling Categorical Attributes

In [ ]:
cat_cols = [cname for cname in train_set.columns if train_set[cname].dtype not in ['int64', 'float64']]
print(cat_cols)

In [ ]:
X_train['climateregions__climateregion'].value_counts()

In [ ]:
test_data['climateregions__climateregion'].value_counts()

In [ ]:
X_train['climateregions__climateregion'].isin(test_data['climateregions__climateregion']).value_counts()

**train_data and test_data have the same climate regions**

In [ ]:
X_train['location'].value_counts()

In [ ]:
test_data['location'].value_counts()

In [ ]:
X_train['location'].isin(test_data['location']).value_counts()

**train_data and test_data have the same locations**

In [ ]:
# encoding Categorical data
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder() # Use ordinal encoder
X_train[cat_cols] = enc.fit_transform(X_train[cat_cols] )
test_data[cat_cols] = enc.transform(test_data[cat_cols] )

### Feature Selection using the Filter Method

In [ ]:
# Using Pearson Correlation
corr_matrix = X_train.corr().abs()

# Select upper triangle of correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [ ]:
# Find features with correlation greater than 0.95
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] >= 0.95)]
print(to_drop)

In [ ]:
# Dropped highly correlated features from the dataset
X_train = X_train.drop(to_drop, axis=1)
X_train.head()

### Explore the Numerical Attributes

In [ ]:
num_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

not_to_scale = {'climateregions__climateregion'}

num_cols = [ele for ele in num_cols if ele not in not_to_scale]
data_num = X_train[num_cols]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
sc = StandardScaler()
mmsc = MinMaxScaler()
X_train[num_cols] = sc.fit_transform(X_train[num_cols] )

In [ ]:
X_train

## Train Dataset

### Adding Datetime features

In [ ]:
dtf = DatetimeFeatures(
        # the datetime variable
        variables ="index",

        # The features we want to create
        features_to_extract=[
            "year",
            "month",
        ]
)
# Extract the datetime features
X_train = dtf.fit_transform(X_train)

### Adding Periodic Features

In [ ]:
# Create features that capture the cyclical representation

cyclicf = CyclicalFeatures(
            variables = ["year", "month"],
            drop_original = True,
)

X_train = cyclicf.fit_transform(X_train)
X_train["season"] = X_train.index.month%12 // 3 + 1

X_train[[ v for v in X_train.columns if "year" in v or "month" in v]]. head()

### Adding Dummie Variables

In [ ]:
# Adding Dummie Variables for the season
X_train = pd.get_dummies(data=X_train, columns=['season'])

In [ ]:
# Adding Dummie Variables for the climateregions
#X_train = pd.get_dummies(data=X_train, columns=['climateregions__climateregion'])

In [ ]:
X_train.head()

## Validation Data Preparation

In [ ]:
# Get the same shape as train set by dropping the less useful features
X_valid = X_valid.drop(to_drop, axis=1)

In [ ]:
# encoding Categorical data
X_valid[cat_cols] = enc.transform(X_valid[cat_cols] )

In [ ]:
val_num_cols = [cname for cname in X_valid.columns if X_valid[cname].dtype in ['int64', 'float64']]

not_to_scale = {'climateregions__climateregion'}

val_num_cols = [ele for ele in val_num_cols  if ele not in not_to_scale]
val_data_num = X_valid[val_num_cols]

In [ ]:
# Scaling numerical data
X_valid[val_num_cols ] = sc.transform(X_valid[val_num_cols] )

In [ ]:
# Extract the datetime features
X_valid = dtf.transform(X_valid)

In [ ]:
# Create features that capture the cyclical representation
X_valid = cyclicf.transform(X_valid)
X_valid["season"] = X_valid.index.month%12 // 3 + 1

X_valid[[ v for v in X_valid.columns if "year" in v or "month" in v]]. head()

In [ ]:
X_valid.index.min(), X_valid.index.max()

Well, I faced a problem here. My validation dataset does not have data for seasons 1, and season 4. My automatic function did not fulfill my requirement as I wanted. So I just created those variables manually. :)

In [ ]:
# Adding Dummie Variables for the season
X_valid = pd.get_dummies(data=X_valid, columns=['season'])
X_valid.insert(loc = len(X_valid.columns)-2, column = 'season_1',value = 0)
X_valid['season_4'] = 0

In [ ]:
# Adding Dummie Variables for the climateregions
# X_valid = pd.get_dummies(data=X_valid, columns=['climateregions__climateregion'])

In [ ]:
X_valid.head()

## Test Data Preparation

In [ ]:
test_data = test_data.drop(to_drop, axis=1)

In [ ]:
test_num_cols = [cname for cname in test_data.columns if test_data[cname].dtype in ['int64', 'float64']]

not_to_scale = {'climateregions__climateregion'}

test_num_cols = [ele for ele in test_num_cols  if ele not in not_to_scale]
test_data_num = test_data[test_num_cols]

In [ ]:
test_data[test_num_cols ] = sc.transform(test_data[test_num_cols] )

In [ ]:
test_data = dtf.transform(test_data)

In [ ]:
test_data = cyclicf.transform(test_data)
test_data["season"] = test_data.index.month%12 // 3 + 1

test_data[[ v for v in test_data.columns if "year" in v or "month" in v]]. head()

In [ ]:
test_data = pd.get_dummies(data=test_data, columns=['season'])
test_data.insert(loc = len(test_data.columns)-1, column = 'season_2',value = 0)
test_data.insert(loc = len(test_data.columns)-1, column = 'season_3',value = 0)

In [ ]:
# test_data = pd.get_dummies(data=test_data, columns=['climateregions__climateregion'])

In [ ]:
test_data.head()

<a id="9"></a> <br>
# 9. Time Series Split for Cross Validation

**Since the dataset is a time-series event log (temperature), I used time-sensitive cross-validation splitter to evaluate the temperature forecasting model as realistically as possible. I used a gap of 3 months between the train and test side of the splits. I also limit the training set size to make the performance of the CV folds more stable**

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_validate

tscv =TimeSeriesSplit(gap=120, max_train_size=10000, n_splits=5, test_size=1000)
#for i, (train_index, test_index) in enumerate(tscv.split(X_train, y_train)):
    #print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")

<a id="10"></a> <br>
# 10. Training, Evaluating and Validation

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

## Fixed a Baseline

In [ ]:
# "Learn" the mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the validation set
baseline_predictions = np.ones(y_valid.shape) * mean_train
# Compute RMSE and MAE
mae_baseline = mean_absolute_error(y_valid, baseline_predictions)
rmse_baseline = np.sqrt(mean_squared_error(y_valid, baseline_predictions))
print("Baseline MAE is {:.2f}".format(mae_baseline))
print("Baseline RMSE is {:.2f}".format(rmse_baseline))

**This scores are what we can achieve with no efforts. Let's see how good my model is.......**

## Gradient Boosting with Ensemble Learning

### Histogram-Based Gradient Boosting

In [ ]:
# This was not the improved model.
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

hgbr = HistGradientBoostingRegressor(loss='squared_error',
                                     learning_rate=0.01,
                                     max_iter=6000,
                                     max_leaf_nodes=31,
                                     max_depth=None,
                                     min_samples_leaf=20,
                                     l2_regularization=1.0,
                                     max_bins=255,
                                     warm_start=False,
                                     early_stopping='auto',
                                     scoring='loss',
                                     validation_fraction=0.1,
                                     n_iter_no_change=10,
                                     tol=1e-07,
                                     verbose=0,
                                     random_state=None)

In [ ]:
hgbr.fit(X=X_train, y=y_train)

In [ ]:
train_predictions = hgbr.predict(X_train)

In [ ]:
# Evaluation on Validation dataset
train_rmse = np.sqrt(mean_squared_error(y_true=y_train, y_pred=train_predictions))
train_r2 = r2_score(y_train, y_pred=train_predictions)
print('rmse value for hgbr model is', train_rmse)
print('r2 value for hgbr model is', train_r2)

In [ ]:
hgbr_predictions = hgbr.predict(X_valid)

In [ ]:
# Evaluation on Validation dataset
valid_rmse = np.sqrt(mean_squared_error(y_true=y_valid, y_pred=hgbr_predictions))
valid_r2 = r2_score(y_valid, y_pred=hgbr_predictions)
print('rmse value for hgbr model is', valid_rmse)
print('r2 value for hgbr model is', valid_r2)

In [ ]:
# Cross Validation HistGradientBoostingRegressor
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )


evaluate(hgbr, X_train, y_train, cv=tscv)

### XGBoost

In [ ]:
import xgboost as xgb

xgbr = xgb.XGBRegressor(learning_rate = 0.2,
                        n_estimators= 8000,
                        max_depth= 5,
                        subsample= 0.8,
                        colsample_bytree= 1,
                        gamma= 1,
                       )

In [ ]:
xgbr.fit(X=X_train, y=y_train, early_stopping_rounds=100, eval_set=[(X_valid, y_valid)], verbose=0)

In [ ]:
# Evaluation on Training dataset
train_predictions = xgbr.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_true=y_train, y_pred=train_predictions))
train_r2 = r2_score(y_train, y_pred=train_predictions)
print('rmse value for xgbr model is', train_rmse)
print('r2 value for xgbr model is', train_r2)

In [ ]:
# Evaluation on Validation dataset
xgbr_predictions = xgbr.predict(X_valid)
valid_rmse = np.sqrt(mean_squared_error(y_true=y_valid, y_pred=xgbr_predictions))
valid_r2 = r2_score(y_valid, y_pred=xgbr_predictions)
print('rmse value for xgbr model is', valid_rmse)
print('r2 value for xgbr model is', valid_r2)

In [ ]:
# Cross Validation XGBoostRegressor
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )


#evaluate(xgbr, X_train, y_train, cv=tscv)

### LightGBM

In [ ]:
from lightgbm import LGBMRegressor
lgbmr = LGBMRegressor(
        objective='regression',
        metric='l1',
        is_unbalance=True,
        bagging_freq=5,
        n_estimators=600,
        boosting='dart',
        num_leaves=28,
        max_depth=12,
        learning_rate=0.1,
        feature_fraction = 0.9,
        num_iterations =10000,
        subsample=0.2)

In [ ]:
lgbmr.fit(X=X_train, y=y_train, early_stopping_rounds=200, eval_set=[(X_valid, y_valid)], verbose=0)

In [ ]:
# Evaluation on Training dataset
train_predictions = lgbmr.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_true=y_train, y_pred=train_predictions))
train_r2 = r2_score(y_train, y_pred=train_predictions)
print('rmse value for lgbmr model is', train_rmse)
print('r2 value for lgbmr model is', train_r2)

In [ ]:
# Evaluation on Validation dataset
lgbmr_predictions = lgbmr.predict(X_valid)
valid_rmse = np.sqrt(mean_squared_error(y_true=y_valid, y_pred=lgbmr_predictions))
valid_r2 = r2_score(y_valid, y_pred=lgbmr_predictions)
print('rmse value for lgbmr model is', valid_rmse)
print('r2 value for lgbmr model is', valid_r2)

In [ ]:
# Cross Validation LightGBMRegressor
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )


#evaluate(lgbmr, X_train, y_train, cv=tscv)

In [ ]:
last_month = slice(-30, None)
fig, ax = plt.subplots(figsize=(12, 4))
fig.suptitle("Predictions for the last 30 days period")
ax.plot(
    y_valid.values[last_month],
    "x-",
    alpha=0.2,
    label="Actual temp",
    color="black",
)
ax.plot(hgbr_predictions[last_month], "x-", label="hgbr-predictions")
ax.plot(xgbr_predictions[last_month], "x-", label="xgbr-predictions")
ax.plot(lgbmr_predictions[last_month], "x-", label="lgbmr-predictions")
_ = ax.legend()

In [ ]:
test_predictions = lgbmr.predict(test_data)

NameError: name 'lgbmr' is not defined

In [ ]:
submit['contest-tmp2m-14d__tmp2m'] = test_predictions
submit.to_csv('submission_lgmbr.csv' ,index = False)